In [1]:
#!/usr/bin/env python3
## import libraries
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, TimeDistributed, RepeatVector
import matplotlib.pyplot as plt
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.regularizers import L1L2

Using TensorFlow backend.
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lab606a/.virtualenvs

In [2]:
# set GPU memory
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# specify
name = 'back6'

n_step = 9

In [3]:
def split(data, depth):
    dataset = data
    depth = depth
    X = np.zeros([int(depth), n_step, 3])
    Y = np.zeros([int(depth), n_step, 3])
    c = 0
    d = 0
    
    for i in range(int(depth)):
        for j in range(n_step):
            if d < dataset.shape[0]:
                X[i,j,:] = dataset[d, c:c+3]
                Y[i,j,:] = dataset[d, (c+3*n_step):(c+3*n_step+3)]
                
                if ((c+3*n_step+3) != (dataset.shape[1])):
                    c +=3
                else:
                    c = 0
                    d += 1
        if (c-3) > 0:
            c = (c - 3*n_step + 3)
        else:
            c = c
            
    return X, Y

In [4]:
path = '/home/lab606a/Documents/20200331/fixed/original data 60hz/' + name + '.csv'
print('read dataset ' + path)
dataset = pd.read_csv(path, header=None)
dataset = dataset.fillna(0)
dataset = np.array(dataset)
dataset.shape

maxlen_train = dataset.shape[1]+((n_step-1)*3)

dataset = sequence.pad_sequences(dataset, maxlen=maxlen_train, padding='post', dtype='float32')
dataset.shape

depth_train = (int(dataset.shape[1]/3)+1-n_step-n_step)*dataset.shape[0] # (all_balls + 1 - input_balls - output_balls)*n_rows

x_train, y_train = split(data=dataset, depth=depth_train)

read dataset /home/lab606a/Documents/20200331/fixed/original data 60hz/back6.csv


In [13]:
model = Sequential()
model.add(LSTM(256, activation='linear', input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(RepeatVector(x_train.shape[1]))
model.add(LSTM(256, activation='linear', return_sequences=True))
model.add(TimeDistributed(Dense(3)))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [14]:
history = model.fit(x_train, y_train, batch_size=3000, epochs=500, shuffle=True)

Epoch 1/500
26490/26490 [==============================] - 2s 67us/step - loss: 2431.9750 - acc: 0.5209
Epoch 2/500
26490/26490 [==============================] - 1s 20us/step - loss: 1795.5351 - acc: 0.6824
Epoch 3/500
26490/26490 [==============================] - 1s 20us/step - loss: 1667.4347 - acc: 0.6485
Epoch 4/500
26490/26490 [==============================] - 1s 20us/step - loss: 1732.6422 - acc: 0.6371
Epoch 5/500
26490/26490 [==============================] - 1s 20us/step - loss: 2185.6331 - acc: 0.5733
Epoch 6/500
26490/26490 [==============================] - 1s 20us/step - loss: 2605.3391 - acc: 0.5218
Epoch 7/500
26490/26490 [==============================] - 1s 20us/step - loss: 3874.0472 - acc: 0.4707
Epoch 8/500
26490/26490 [==============================] - 1s 20us/step - loss: 4340.0408 - acc: 0.5451
Epoch 9/500
26490/26490 [==============================] - 1s 20us/step - loss: 5243.9810 - acc: 0.5073
Epoch 10/500
26490/26490 [==============================] - 1s 2

KeyboardInterrupt: 

In [ ]:
# plot accuracy history
# summarize history for accuracy 
plt.plot(history.history['acc'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(x_train[0,:,:])
print(y_train[0,:,:])

In [ ]:
y_pred = model.predict(x_train[0,:,:].reshape(1,9,3))
y_pred

In [ ]:
error = y_train[0,:,:].reshape(1,9,3)-y_pred
error